# Шифр «Поворотная решетка»

## Функции для работы

In [1]:
import math
import random
import itertools
import numpy as np

EMPTY = 2
HOLE = 1
BLOCK = 0


def getAllDims(value):
    dims = []
    for i in range(1, int(math.sqrt(value) + 1)):
        j = value // i
        if value % i == 0 and j % 2 == 0 and i % 2 == 0:
            dims.append([i, j])
    return dims


def getAllDimsText(text):
    value = len(text)
    dims = []
    for i in range(1, int(math.sqrt(value) + 1)):
        j = value // i
        if value % i == 0 and j % 2 == 0 and i % 2 == 0:
            dims.append([i, j])
    return dims


def testDim(pair):
    return pair[0] % 2 == 0 and pair[1] % 2 == 0


def testDimsForText(text, dims):
    l = len(text)
    r = 0
    for dim in dims:
        r += dim[0]*dim[1]
    if l > r:
        return False, [l, r]
    else:
        return True, [l, r]
    
    
def writeDimsToFile(path, pairs):
    with open(path, 'w') as file:
        for pair in pairs:
            file.write(f"{pair[0]}x{pair[1]}\n")


def readDimsFromFile(path):
    pairs = []
    with open(path, 'r') as file:
        for line in file:
            pairs.append(list(map(int, line.split('x'))))
    return pairs


def writeKeysToFile(path, keys):
    with open(path, 'w') as file:
        for grille in keys:
            grille_str = "".join(map(str, grille.flatten()))
            file.write(f"{grille.shape[0]}x{grille.shape[1]}x{grille_str}\n")


def readKeysFromFile(path):
    keys = []
    with open(path, 'r') as file:
        for line in file:
            items = line.strip().split('x')
            grille = np.array(tuple(items[2]), dtype=int).reshape(
                int(items[0]), int(items[1]))
            keys.append(grille)
    return keys


def genGrilleRandom(dim):
    grille = np.zeros(dim[0] * dim[1], dtype=int).reshape(dim[0], dim[1])
    all_index = list(itertools.product(range(0, dim[0]), range(0, dim[1])))

    while len(all_index) != 0:
        i = random.randint(0, len(all_index) - 1)

        ind_1 = all_index[i]
        ind_2 = dim[0] - 1 - all_index[i][0], all_index[i][1]
        ind_3 = dim[0] - 1 - all_index[i][0], dim[1] - 1 - all_index[i][1]
        ind_4 = all_index[i][0], dim[1] - 1 - all_index[i][1]

        grille[ind_1[0]][ind_1[1]] = HOLE
        grille[ind_2[0]][ind_2[1]] = BLOCK
        grille[ind_3[0]][ind_3[1]] = BLOCK
        grille[ind_4[0]][ind_4[1]] = BLOCK

        all_index.pop(all_index.index(ind_1))
        all_index.pop(all_index.index(ind_2))
        all_index.pop(all_index.index(ind_3))
        all_index.pop(all_index.index(ind_4))
    return grille


def encode(text, grille):
    encode_matrix = np.chararray(grille.shape, unicode=True)
    encode_matrix[:] = ''
    text_index = 0
    # Прямое
    for m_index, value in np.ndenumerate(grille):
        if value == HOLE:
            encode_matrix[m_index] = text[text_index]
            text_index += 1
    # Переворот на 180
    rot180 = np.rot90(grille, 2)
    for m_index, value in np.ndenumerate(rot180):
        if value == HOLE:
            encode_matrix[m_index] = text[text_index]
            text_index += 1
    # Отзеркаливание с переворотом
    flip_rot180 = np.fliplr(rot180)
    for m_index, value in np.ndenumerate(flip_rot180):
        if value == HOLE:
            encode_matrix[m_index] = text[text_index]
            text_index += 1
    # Поворот на 180 отзеркаливания с переворотом
    rot180_flip_rot180 = np.rot90(flip_rot180, 2)
    for m_index, value in np.ndenumerate(rot180_flip_rot180):
        if value == HOLE:
            encode_matrix[m_index] = text[text_index]
            text_index += 1
    return encode_matrix


def decode(matrix, grille):
    decode_text = []
    # Прямое
    for m_index, value in np.ndenumerate(grille):
        if value == HOLE:
            decode_text.append(matrix[m_index])
    # Переворот на 180
    rot180 = np.rot90(grille, 2)
    for m_index, value in np.ndenumerate(rot180):
        if value == HOLE:
            decode_text.append(matrix[m_index])
    # Отзеркаливание с переворотом
    flip_rot180 = np.fliplr(rot180)
    for m_index, value in np.ndenumerate(flip_rot180):
        if value == HOLE:
            decode_text.append(matrix[m_index])
    # Поворот на 180 отзеркаливания с переворотом
    rot180_flip_rot180 = np.rot90(flip_rot180, 2)
    for m_index, value in np.ndenumerate(rot180_flip_rot180):
        if value == HOLE:
            decode_text.append(matrix[m_index])
    return ''.join(decode_text)


def encodeWithOneKey(text, key):
    return ''.join(list(encode(text, key).flatten()))


def decodeWithOneKey(text, key):
    matrix = np.array(list(text)).reshape(key.shape)
    return ''.join(decode(matrix, key))


def encodeWithSeveralKeys(text, keys, aggreg="x"):
    text = text.replace(' ', '').replace('\t', '').replace('\n', '')

    encode_result = []
    ptr = 0
    for key in keys:
        while(ptr + key.size > len(text)):
            text += aggreg
        part_text = text[ptr: ptr + key.size]
        temp = encode(part_text, key)
        encode_result += list(temp.flatten())
        ptr += key.size
    return ''.join(encode_result)


def decodeWithSeveralKeys(text, keys):
    decode_result = []
    ptr = 0
    for key in keys:
        part_text = text[ptr: ptr + key.size]
        matrix = np.array(list(part_text)).reshape(key.shape)
        decode_result += decode(matrix, key)
        ptr += key.size
    return ''.join(decode_result)


def prepareTextForOneKey(text, aggreg="x"):
    text = text.replace(' ', '').replace('\t', '').replace('\n', '')
    while(len(text) % 4 != 0):
        text = text + aggreg
    return text


def prepareTextForSeveralKey(text):
    text = text.replace(' ', '').replace('\t', '').replace('\n', '')
    return text

## Использование

In [2]:
textToEncode = "ПОВОРОТНАЯ ШИФРРЕШЕТКА ЯВЛЯЕТСЯ ЧАСТНЫМ СЛУЧАЕМ \nШИФРА МАРШРУТНОЙ ПЕРЕСТАНОВКИ"
print(textToEncode)

ПОВОРОТНАЯ ШИФРРЕШЕТКА ЯВЛЯЕТСЯ ЧАСТНЫМ СЛУЧАЕМ 
ШИФРА МАРШРУТНОЙ ПЕРЕСТАНОВКИ


### Шифрование с одним ключем

In [3]:
# Убирание переносов, пробелов и табуляции
# Дополнение строки для кратности 4м
tmp = prepareTextForOneKey(textToEncode)
print(tmp)
print("Длина сообщения:",len(tmp))

ПОВОРОТНАЯШИФРРЕШЕТКАЯВЛЯЕТСЯЧАСТНЫМСЛУЧАЕМШИФРАМАРШРУТНОЙПЕРЕСТАНОВКИxx
Длина сообщения: 72


In [4]:
# Вывод возможных ключей
dims = getAllDimsText(tmp)
for i, dim in enumerate(dims):
    print(f"{i}: {dim}")

0: [2, 36]
1: [4, 18]
2: [6, 12]


In [5]:
# Выбираем размерность [4, 18]
dim = dims[1]

# Генерируем случайныйй ключ
key = genGrilleRandom(dim)
print(key)

# Запись ключа в файл
writeKeysToFile("key.txt", [key])

[[0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
# Шифруем текст одним ключем
text_encode = encodeWithOneKey(tmp, key)
print(text_encode)

СПЛОТУВНООРЙТОПКЕАЯТЧНАРЕВЕЛМАЯЯСЕТШАИШФИТСНФОРРАВЯКЧРЕМШААЕРСТШРНИУЫxМx


### Расшифровка одним ключем

In [7]:
# Чтение ключа из файла
tmp_keys = readKeysFromFile("key.txt")
key = tmp_keys[0]
print(key)

[[0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [8]:
# Расшифровка
text_decode = decodeWithOneKey(text_encode, key)
print(text_decode, "\n")
print(textToEncode)

ПОВОРОТНАЯШИФРРЕШЕТКАЯВЛЯЕТСЯЧАСТНЫМСЛУЧАЕМШИФРАМАРШРУТНОЙПЕРЕСТАНОВКИxx 

ПОВОРОТНАЯ ШИФРРЕШЕТКА ЯВЛЯЕТСЯ ЧАСТНЫМ СЛУЧАЕМ 
ШИФРА МАРШРУТНОЙ ПЕРЕСТАНОВКИ


---
### Шифрование с несколькими ключами

In [9]:
# Создаем массив желаемых размерностей ключа
key_dims = [[4, 6], [6, 12]]

# Проверка размерностей
for dim in key_dims:
    if not testDim(dim):
        print(f"Неверная размерность ключа:{dim}")
        
t, s = testDimsForText(textToEncode, key_dims)
if not t:
    print(f"Не возможно зашифровать текст. Длина текста: {s[0]}, суммарная длина ключей: {s[1]}")
else:
    print(f"Длина текста: {s[0]}, суммарная длина ключей: {s[1]}")

Длина текста: 78, суммарная длина ключей: 96


In [10]:
# Генерация случайных ключей
keys = [genGrilleRandom(dim) for dim in key_dims]

for key in keys:
    print(key)
    
# Запись ключей в файл
writeKeysToFile("key.txt", keys)

[[0 0 0 1 0 0]
 [0 0 1 0 1 0]
 [0 0 0 0 0 1]
 [1 1 0 0 0 0]]
[[1 1 0 1 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]]


In [11]:
# Шифруем текст
text_encode = encodeWithSeveralKeys(textToEncode, keys)
print(text_encode)

ФРТПТНАКОАВРЯЯЕШШОРОИЕВЛЯЕxТxРМСxЯxxЧЕxШАИСxТСФxРxxТxНxЫxМСАxАМНАОРВШКИЛxУРxxxЧУАxxТxxНxОЕxxЙxПЕ


### Расшифровка с несколькими ключами

In [12]:
# Чтение ключей из файла
keys = readKeysFromFile("key.txt")
for key in keys:
    print(key)

[[0 0 0 1 0 0]
 [0 0 1 0 1 0]
 [0 0 0 0 0 1]
 [1 1 0 0 0 0]]
[[1 1 0 1 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]]


In [13]:
# Расшифровка
text_decode = decodeWithSeveralKeys(text_encode, keys)
print(text_decode, "\n")
print(textToEncode)

ПОВОРОТНАЯШИФРРЕШЕТКАЯВЛЯЕТСЯЧАСТНЫМСЛУЧАЕМШИФРАМАРШРУТНОЙПЕРЕСТАНОВКИxxxxxxxxxxxxxxxxxxxxxxxxxx 

ПОВОРОТНАЯ ШИФРРЕШЕТКА ЯВЛЯЕТСЯ ЧАСТНЫМ СЛУЧАЕМ 
ШИФРА МАРШРУТНОЙ ПЕРЕСТАНОВКИ
